## 684. Redundant Connection


In this problem, a tree is an **undirected graph** that is connected and has no cycles.

You are given a graph that started as a tree with `n` nodes labeled from `1` to `n`, with one additional edge added. The added edge has two **different** vertices chosen from `1` to `n`, and was not an edge that already existed. The graph is represented as an array `edges` of length `n` where `edges[i] = [a<sub>i</sub>, b<sub>i</sub>]` indicates that there is an edge between nodes `a<sub>i</sub>` and `b<sub>i</sub>` in the graph.

Return _an edge that can be removed so that the resulting graph is a tree of_ `n` _nodes_. If there are multiple answers, return the answer that occurs last in the input.

**Example 1:**

![](https://assets.leetcode.com/uploads/2021/05/02/reduntant1-1-graph.jpg)

```
Input: edges = [[1,2],[1,3],[2,3]]
Output: [2,3]

```

**Example 2:**

![](https://assets.leetcode.com/uploads/2021/05/02/reduntant1-2-graph.jpg)

```
Input: edges = [[1,2],[2,3],[3,4],[1,4],[1,5]]
Output: [1,4]

```

**Constraints:**

-   `n == edges.length`
-   `3 <= n <= 1000`
-   `edges[i].length == 2`
-   `1 <= a<sub>i</sub> < b<sub>i</sub> <= edges.length`
-   `a<sub>i</sub> != b<sub>i</sub>`
-   There are no repeated edges.
-   The given graph is connected.

---


- URL: [Problem_URL](https://leetcode.com/problems/redundant-connection/description/)

- Topics: Union Find, DFS, BFS

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, Most Optimum, Union Find
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
class UnionFind:
    def __init__(self, n):
        self.parent = [i for i in range(n)]
        self.size = [1 for _ in range(n)]

    def find(self, node):
        if self.parent[node] != node:
            self.parent[node] = self.find(self.parent[node])
        
        return self.parent[node]
    
    def union(self, nodeA, nodeB):
        parentA = self.find(nodeA)
        parentB = self.find(nodeB)

        if parentA == parentB:
            return False
        else:
            if self.size[parentA] < self.size[parentB]:
                self.parent[parentA] = parentB
                self.size[parentB] += self.size[parentA]
            else:
                self.parent[parentB] = parentA
                self.size[parentA] += self.size[parentB]
            
            return True


class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        uf = UnionFind(len(edges))

        for nodeA, nodeB in edges:
            if not uf.union(nodeA-1, nodeB-1):
                return [nodeA, nodeB]
        
        return []

### Solution 2, DFS with parent and visited array to detect cycle.

Here, N is the number of nodes and edges in the given graph.

- Time complexity: O(N).

    We perform the DFS starting from node 0 only once, which has a time complexity of O(N). Then, we iterate over the cycle nodes using the parent array, with a maximum of N iterations if all nodes are part of the cycle. Finally, we iterate over all edges and check the map in O(1) time for each edge. Therefore, the total time complexity is O(N).

- Space complexity: O(N)

    The adjacency list adjList will store N edges, and the size of the visited array is N. Additionally, space is required for the active stack calls during DFS, which can be as large as one per node. The map cycleNodes can contain at most N entries. Therefore, the total space complexity is O(N).


In [ ]:
class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        n = len(edges)
        adj = [[] for _ in range(n)]
        for u, v in edges:
            adj[u - 1].append(v - 1)
            adj[v - 1].append(u - 1)

        visited = [False] * n
        parent = [-1] * n
        cycle_start = -1
        cycle_end = -1

        def dfs(node, par):
            nonlocal cycle_start, cycle_end
            visited[node] = True
            for neighbor in adj[node]:
                if neighbor == par:
                    continue
                if visited[neighbor]:
                    # Back edge found: cycle from neighbor -> ... -> node
                    cycle_start = neighbor
                    cycle_end = node
                    return True
                parent[neighbor] = node
                if dfs(neighbor, node):
                    return True
            return False

        # Run DFS until cycle is found
        dfs(0, -1)

        # Reconstruct the cycle path
        cycle_nodes = set()
        # Go from cycle_end up to cycle_start using parent pointers
        current = cycle_end
        while current != cycle_start:
            cycle_nodes.add(current)
            current = parent[current]
        cycle_nodes.add(cycle_start)

        # Traverse edges in reverse order; first one with both ends in cycle is answer
        for i in range(len(edges) - 1, -1, -1):
            u, v = edges[i][0] - 1, edges[i][1] - 1
            if u in cycle_nodes and v in cycle_nodes:
                return edges[i]

        return []  # Should never reach here for valid input